In [55]:
## Working Notebook to add confeernce affilation to game data table

# Dependencies

import pandas as pd
import re
import numpy as np
import ast

# Load Data
conference_df = pd.read_csv('../data/yearly_conference_members.csv')
games_df = pd.read_csv('../data/cfb_scores_all_years.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})



# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


C:\Users\Justin\AppData\Local\Temp\ipykernel_10676\433976151.py:12: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('../data/cfb_scores_all_years.csv')


In [56]:
conference_df.sample(5)

,Year,American Athletic Conference,ACC,Big Ten Conference,Big 12,Big 8 Conference,Big East Conference,Border Conference,Conference USA,Ivy League,Mid-American Conference,Missouri Valley Conference,Mountain West Conference,Pac-12 Conference,Pacific Coast Conference,Sun Belt Conference,SEC,Skyline Conference,Southwest Conference,Western Athletic Conference
128,2024,"[UAB, East Carolina, Florida Atlantic, Memphis...","[Clemson, Duke, North Carolina, NC State, Wake...","[Indiana, Maryland, Michigan State, Ohio State...","[BYU, Central Florida, Cincinnati, Houston]",[],"[Cincinnati, Connecticut, Louisville, Pittsbur...",[],"[Florida International, Jacksonville State, Li...",[],"[Akron, Bowling Green, Buffalo, Kent State, Mi...",[],"[United States Air Force Academy, Boise State,...",[],[],"[Appalachian State, Arkansas State, Coastal Ca...","[Florida, Georgia, Kentucky, South Carolina, T...",[],[],"[Boise State, Fresno State, Hawaii, Idaho, Lou..."
87,1983,[],"[Clemson, Duke, North Carolina, NC State, Wake...","[Indiana, Michigan State, Ohio State, Illinois...",[],"[Colorado, Iowa State, Kansas, Kansas State, M...","[Connecticut, Pittsburgh, Syracuse, Boston Col...",[],[],[],"[Bowling Green, Kent State, Miami (OH), Ohio, ...","[Creighton, Tulsa, West Texas State, Wichita S...",[],"[Arizona, Arizona State, California, UCLA, Ore...",[],"[Georgia State, Old Dominion, South Alabama, A...","[Florida, Georgia, Kentucky, Tennessee, Vander...",[],"[Arkansas, Texas A&M, Texas, Baylor, Texas Tec...","[Hawaii, San Diego State]"
55,1951,[],[],"[Indiana, Michigan State, Ohio State, Illinois...",[],"[Colorado, Iowa State, Kansas, Kansas State, M...",[],"[Arizona, Arizona State, Hardin–Simmons, New M...",[],[],"[Kent State, Miami (OH), Ohio, Toledo, Western...","[Oklahoma A&M, Saint Louis, Detroit, Houston, ...",[],[],"[California, Oregon, Oregon State, Washington,...",[],"[Florida, Georgia, Kentucky, Tennessee, Vander...","[BYU, Colorado State, Denver, Montana, New Mex...","[Arkansas, Texas A&M, Texas, Baylor, TCU, Rice...",[]
90,1986,[],"[Clemson, Duke, North Carolina, NC State, Wake...","[Indiana, Michigan State, Ohio State, Illinois...",[],"[Colorado, Iowa State, Kansas, Kansas State, M...","[Connecticut, Pittsburgh, Syracuse, Boston Col...",[],[],[],"[Bowling Green, Kent State, Miami (OH), Ohio, ...","[Creighton, Tulsa, Wichita State]",[],"[Arizona, Arizona State, California, UCLA, Ore...",[],"[Georgia State, Old Dominion, South Alabama, A...","[Florida, Georgia, Kentucky, Tennessee, Vander...",[],"[Arkansas, Texas A&M, Texas, Baylor, Texas Tec...","[Hawaii, San Diego State]"
28,1924,[],[],"[Indiana, Ohio State, Illinois, Iowa, Minnesot...",[],"[Iowa State, Kansas, Kansas State, Missouri, N...",[],[],[],[],[],"[Grinnell, Iowa State, Kansas State, Kansas, M...",[],[],"[California, Oregon, Oregon State, Washington,...",[],[],[],"[Oklahoma A&M, Arkansas, Texas A&M, Texas, Bay...",[]


In [57]:
# Function to retrieve the conference based on year and team name
def get_conference_adjusted(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Create a lookup dictionary for team-year to conference mapping
conference_lookup = {}

for _, row in conference_df.iterrows():
    year = row['Year']
    for col, teams in row[1:].items():
        for team in teams:
            conference_lookup[(year, team)] = col

# Use the adjusted function to assign conference for Winner and Loser
games_df['winner_conference'] = games_df.apply(lambda row: get_conference_adjusted(row['Year'], row['Winner'], conference_lookup), axis=1)
games_df['loser_conference'] = games_df.apply(lambda row: get_conference_adjusted(row['Year'], row['Loser'], conference_lookup), axis=1)

# Classify games as Regular Season or Post Season based on Date
games_df['Date'] = pd.to_datetime(games_df['Date'], errors='coerce')  # Convert the Date column to datetime format
games_df['season_type'] = games_df.apply(lambda row: "Post Season" if (row['Date'].month == 12 and row['Date'].day > 15) or row['Date'].month == 1 else "Regular Season", axis=1)


In [58]:
# games_df.info()

# value counts for winner and loser conference
games_df['winner_conference'].value_counts()
games_df['loser_conference'].value_counts()

# OUTPUT CSV TO TEMP FOLDER
games_df.to_csv('../TEMP/ADJUSTED_cfb_scores_all_years.csv', index=False)

# Load The FBS Teams Table
fbs_teams_df = pd.read_csv('..\data\cfb_d1_teams_with_coordinates.csv')

In [59]:
games_df.head()
games_df.columns

Index(['Rk', 'Wk', 'Date', 'Day', 'Winner', 'Pts', 'Unnamed: 6', 'Loser',
       'Pts.1', 'Notes', 'Year', 'Time', 'Unnamed: 7', 'Winner_Ranking',
       'Loser_Ranking', 'winner_conference', 'loser_conference',
       'season_type'],
      dtype='object')

In [60]:
adjusted_games_df = games_df

# Extract the list of top division teams
d1_teams_list = fbs_teams_df['Team'].tolist()

# Filter the historical game data to only include games involving these teams
filtered_games_df = adjusted_games_df[(adjusted_games_df['Winner'].isin(d1_teams_list)) | (adjusted_games_df['Loser'].isin(d1_teams_list))]

# Display basic information and a sample of rows from the filtered dataframe
filtered_games_info = filtered_games_df.info()
filtered_games_sample = filtered_games_df.sample(30)

filtered_games_info, filtered_games_sample


<class 'pandas.core.frame.DataFrame'>
Int64Index: 65508 entries, 0 to 81603
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Rk                 65508 non-null  int64         
 1   Wk                 65508 non-null  int64         
 2   Date               65508 non-null  datetime64[ns]
 3   Day                65508 non-null  object        
 4   Winner             65508 non-null  object        
 5   Pts                65507 non-null  float64       
 6   Unnamed: 6         24057 non-null  object        
 7   Loser              65508 non-null  object        
 8   Pts.1              65507 non-null  float64       
 9   Notes              4966 non-null   object        
 10  Year               65508 non-null  int64         
 11  Time               7080 non-null   object        
 12  Unnamed: 7         3440 non-null   object        
 13  Winner_Ranking     0 non-null      float64       
 14  Loser_

(None,
         Rk  Wk       Date  Day                  Winner   Pts Unnamed: 6  \
 31818  405   8 1953-11-07  Sat       Mississippi State  21.0          @   
 26302  389  12 1944-11-11  Sat          Oklahoma State  13.0          @   
 42489  673  14 1970-11-28  Sat                  Hawaii  49.0        NaN   
 8559   246   6 1914-10-24  Sat              Pittsburgh  21.0        NaN   
 37112  405   8 1962-11-03  Sat               Minnesota  28.0          @   
 50504   30   1 1981-09-05  Sat    Southern Mississippi  33.0        NaN   
 1531    33   5 1895-10-05  Sat                Illinois  48.0        NaN   
 70057  192   3 2009-09-19  Sat    Southern Mississippi  37.0        NaN   
 18567  471   8 1932-11-05  Sat                    Rice  12.0          @   
 1337    26   6 1894-10-06  Sat                 Chicago  46.0        NaN   
 12983  223   5 1923-10-20  Sat                 Alabama   7.0        NaN   
 28434  193   5 1948-10-09  Sat                Maryland  28.0          N   
 1519

In [61]:
# Adjust the conference lookup to assign "Independent/Unknown" for missing teams
def get_conference_adjusted(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Rename df to games_df_new
games_df_new = games_df

# Use the adjusted function to assign conference for Winner and Loser
games_df_new['winner_conference'] = games_df_new.apply(lambda row: get_conference_adjusted(row['Year'], row['Winner'], conference_lookup), axis=1)
games_df_new['loser_conference'] = games_df_new.apply(lambda row: get_conference_adjusted(row['Year'], row['Loser'], conference_lookup), axis=1)

# Classify games as Regular Season or Post Season based on Date
games_df_new['Date'] = pd.to_datetime(games_df_new['Date'], errors='coerce')  # Convert the Date column to datetime format
games_df_new['season_type'] = games_df_new.apply(lambda row: "Post Season" if (row['Date'].month == 12 and row['Date'].day > 15) or row['Date'].month == 1 else "Regular Season", axis=1)



# Classify games as Conference or Non-Conference - Teams in the same conference are considered conference games - Indepentent teams are allway considered non-conference games
games_df_new['game_type'] = games_df_new.apply(lambda row: "Conference" if row['winner_conference'] == row['loser_conference'] else "Non-Conference", axis=1)

# If both winner_conference and loser_conference are "Independent/Unknown", then classify the game as "Non-Conference"
games_df_new['game_type'] = games_df_new.apply(lambda row: "Non-Conference" if row['winner_conference'] == "Independent/Unknown" and row['loser_conference'] == "Independent/Unknown" else "Conference", axis=1)


# Display the updated dataframe with the new columns
games_df_new.sample(30)



,Rk,Wk,Date,Day,Winner,Pts,Unnamed: 6,Loser,Pts.1,Notes,Year,Time,Unnamed: 7,Winner_Ranking,Loser_Ranking,winner_conference,loser_conference,season_type,game_type
25896,372,12,1943-11-27,Sat,Del Monte Pre-Flight,47.0,@,California,8.0,NaN,1943,NaN,NaN,NaN,NaN,Independent/Unknown,Pacific Coast Conference,Regular Season,Conference
14995,421,9,1926-11-13,Sat,Colorado State,3.0,@,Colorado,0.0,NaN,1926,NaN,NaN,NaN,NaN,Independent/Unknown,Independent/Unknown,Regular Season,Non-Conference
4178,58,3,1904-10-01,Sat,Colorado,33.0,@,Utah,6.0,NaN,1904,NaN,NaN,NaN,NaN,Independent/Unknown,Independent/Unknown,Regular Season,Non-Conference
62309,603,13,1998-11-21,Sat,Miami (OH),20.0,@,Akron,14.0,NaN,1998,NaN,NaN,NaN,NaN,Mid-American Conference,Mid-American Conference,Regular Season,Conference
7623,156,5,1912-10-19,Sat,Brown,30.0,NaN,Pennsylvania,7.0,NaN,1912,NaN,NaN,NaN,NaN,Independent/Unknown,Independent/Unknown,Regular Season,Non-Conference
75944,213,4,2016-09-17,Sat,Tennessee,28.0,NaN,Ohio,19.0,"Neyland Stadium - Knoxville, Tennessee",2016,12:00 PM,NaN,NaN,NaN,SEC,Mid-American Conference,Regular Season,Conference
79812,562,18,2021-01-01,Fri,Northwestern,35.0,NaN,Auburn,19.0,"Citrus Bowl (Camping World Stadium - Orlando, ...",2020,1:00 PM,N,NaN,NaN,Big Ten Conference,SEC,Post Season,Conference
20430,297,5,1935-10-19,Sat,Loyola Marymount,13.0,NaN,Arizona,6.0,NaN,1935,NaN,NaN,NaN,NaN,Independent/Unknown,Border Conference,Regular Season,Conference
6155,303,9,1908-11-14,Sat,Arkansas,42.0,NaN,Pittsburg State,12.0,NaN,1908,NaN,NaN,NaN,NaN,Independent/Unknown,Independent/Unknown,Regular Season,Non-Conference
44315,302,7,1973-10-13,Sat,Furman,38.0,NaN,Davidson,7.0,NaN,1973,NaN,NaN,NaN,NaN,Independent/Unknown,Independent/Unknown,Regular Season,Non-Conference


In [62]:
## Extract Notes for Bowl Game and Champ Game Names

# Value COunts
games_df_new['Notes'].value_counts().head(20)

# If notes contains Bowl, then copy the notes to the bowl_game column
games_df_new['bowl_game'] = games_df_new.apply(lambda row: row['Notes'] if isinstance(row['Notes'], str) and 'Bowl' in row['Notes'] else None, axis=1)
# Clear the Bowl Cells from the Notes Column
games_df_new['Notes'] = games_df_new.apply(lambda row: None if isinstance(row['Notes'], str) and 'Bowl' in row['Notes'] else row['Notes'], axis=1)




# value counts for bowl_game
games_df_new['bowl_game'].value_counts()
# Notes value count
games_df_new['Notes'].value_counts()

# Save new Csv to data folder as game_info_v2
games_df_new.to_csv('../data/game_info_v2.csv', index=False)


In [63]:
## value counts for game_type
games_df_new['season_type'].value_counts()


# # # value count for winner_conference
games_df_new['winner_conference'].value_counts()

# # # value count for loser_conference
# games_df['loser_conference'].value_counts()


Independent/Unknown             40425
Big Ten Conference               7079
SEC                              5213
Mid-American Conference          3989
Pac-12 Conference                3907
ACC                              3345
Big 8 Conference                 2821
Southwest Conference             2738
Pacific Coast Conference         1969
Conference USA                   1570
Missouri Valley Conference       1517
Western Athletic Conference      1498
Big East Conference              1336
Ivy League                        987
Sun Belt Conference               984
Border Conference                 726
Skyline Conference                687
Mountain West Conference          675
American Athletic Conference      138
Name: winner_conference, dtype: int64